Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [1]:
from random import choices, randint, random, uniform
from math import exp

import lab9_lib

In [2]:
fitness = lab9_lib.make_problem(10)
for n in range(10):
    ind = choices([0, 1], k=50)
    # here, we print the the genome (as a string) and the fitness function of this genome
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")
    # fitness(ind) passes the genome using the __call__ function
print(fitness.calls)

11010001111100000001111011111111110100001101010101: 19.13%
00100000110100000001011100100001000100011100100000: 7.36%
00100111100000110111100101011001100110010101001000: 7.33%
11111010110000010110000100000000001101011110011100: 11.56%
00000001000001111111100110001101110111011101000110: 15.34%
00010101000111101001110000011110100001001011111000: 23.56%
01101111000001011000010000000001000010100111011110: 15.36%
11000100110110001001111010110000000111000000010101: 29.56%
10111101101011101111010100010010101011100000100000: 15.33%
11011011011001000101100000010000101010011001011001: 15.33%
10


## Local Search Algorithm
This Local Search utilizes the best individual obtained from the Genetic Algorithm. Initially, I set up the Local Search variables. During the algorithm's execution, neighbors are generated by randomly flipping a subset of bits, deviating from the known Simulated Annealing-based approach.

In [3]:
class LocalSearch:
    def __init__(self, problem, initial_solution, iterations=10):
        self.problem = problem
        self.genome_size = len(initial_solution)
        self.best_individual = initial_solution
        self.best_fitness = problem(self.best_individual)
        self.iterations_without_improvement = 0
        self.iterations = iterations

    def run(self):
        for _ in range(self.iterations):
            neighbor = self.generate_neighbor(self.best_individual)
            neighbor_fitness = self.problem(neighbor)

            if neighbor_fitness <= self.best_fitness:
                self.iterations_without_improvement += 1
            else:
                self.best_individual = neighbor
                self.best_fitness = neighbor_fitness
                self.iterations_without_improvement = 0

            if self.best_fitness == 1.0:
                break

    def generate_neighbor(self, individual):
        index_to_mutate = choices(range(self.genome_size), k=self.genome_size)
        neighbor = list(individual)

        # randomly flipping of bits
        mutation_rate = 0.1  # mutation rate of 10%
        for index in index_to_mutate:
            if random() < mutation_rate:
                neighbor[index] = 1 - neighbor[index]

        return neighbor

    @staticmethod
    def acceptance_probability(current_fitness, new_fitness, temperature):
        delta_fitness = new_fitness - current_fitness
        if delta_fitness < 0:
            return 1.0
        else:
            return exp(-delta_fitness / temperature)

## Genetic Algorithm
Here, I define the true genetic algorithm.

In [4]:
class GeneticAlgorithm:
    def __init__(self, problem, genome_size=1000, population_size=50, generations=100, mutation_rate=0.1, local_search_iterations=10, convergence_threshold=5):
        # Setting up Genetic Algorithm parameters
        self.problem = problem
        self.genome_size = genome_size
        self.population_size = population_size
        self.max_generations = generations
        self.mutation_rate = mutation_rate
        self.local_search_iterations = local_search_iterations
        self.convergence_threshold = convergence_threshold

    def run(self):
        best_individual_after_ls = None
        count_no_improvement_steps = 0

        # Population initialization
        population = [choices([0, 1], k=self.genome_size) for _ in range(self.population_size)]

        for generation in range(self.max_generations):
            # Fitness evaluation
            fitness_scores = [self.problem(individual) for individual in population]
            best_individual = population[fitness_scores.index(max(fitness_scores))]

            # Applying local search to the best individual
            local_search = LocalSearch(self.problem, best_individual, iterations=self.local_search_iterations)
            local_search.run()
            best_individual_after_ls = local_search.best_individual
            fitness_after_local = local_search.best_fitness

            # Checking for fitness improvement after local search
            if fitness_after_local > self.problem(best_individual):
                count_no_improvement_steps = 0
            else:
                count_no_improvement_steps += 1

            # Termination condition based on consecutive generations with no improvement
            if count_no_improvement_steps >= self.convergence_threshold:
                break

            # Replacing an individual in the population if fitness improves after local search
            if fitness_after_local > self.problem(best_individual):
                index_to_replace = fitness_scores.index(min(fitness_scores))
                population[index_to_replace] = best_individual_after_ls

            # Updating population using crossover and mutation
            new_population = self.crossover(population)
            self.mutate(new_population)
            population = new_population

        # Returning the best individual after all generations
        return best_individual_after_ls

    def crossover(self, population):
        new_population = []

        for _ in range(self.population_size // 2):
            # Tournament selection of parents
            parent1 = choices(population)[0]
            parent2 = choices(population)[0]

            # Two-point crossover
            crossover_points = sorted([randint(0, self.genome_size - 1) for _ in range(2)])

            child1 = parent1[:crossover_points[0]] + parent2[crossover_points[0]:crossover_points[1]] + parent1[crossover_points[1]:]
            child2 = parent2[:crossover_points[0]] + parent1[crossover_points[0]:crossover_points[1]] + parent2[crossover_points[1]:]

            new_population.extend([child1, child2])

        return new_population

    def mutate(self, population):
        for i in range(len(population)):
            for j in range(self.genome_size):
                if random() < self.mutation_rate:
                    population[i][j] = 1 - population[i][j]



Here there is the main part, in which I call the genetic algorithm and I print all the results.

In [5]:
# Usage of the Genetic Algorithm
instances = [1, 2, 5, 10]

for instance in instances:
    fitness_value = instance
    fitness = lab9_lib.make_problem(fitness_value)
    fitness.calls_increment = fitness_value

    ga = GeneticAlgorithm(problem=fitness, genome_size=1000, population_size=50, generations=100, mutation_rate=0.1, local_search_iterations=10, convergence_threshold=5)
    best_solution = ga.run()

    print(f"Instance {instance}")
    print(f"Best solution: {''.join(map(str, best_solution))}")
    print(f"Best fitness value: {fitness(best_solution):.2%}")
    print(f"Total calls: {fitness.calls}\n")

Instance 1
Best solution: 00001001001110101110100100010111111010110001101000101110110111000110100111011111111010011110001111100011111001111101111101010011101111011110110011001111111111101001010000101110011010000110000001111111011101101100011001111111101011010010000000111101000101011010101111101001100000001110001011111010111111001011100101111010011000101111111011011110101011101101110011111100111000100110010110111110010100101000101100001101100011011001011111001011100001101000110101011010000111111111101110101101101101000100010000111010110011001111101010100111111101111100010000101010111010001100100000001101010111001010010110011011111100101001110110000001010011101001111001110010101100011010011010011111000101011001111101110100011100011000011001001100100111100000111110001100110010011001110000111011110100100100001011111111111101110110111100010001011010101000011101001111010001010101010000100100111110110101011000010011111111001101110011110011010010111111010011110110000110100111110000001001100100